# Scrape Data - IEC World Plugs

In this notebook, we build a list of plug types by country based on the [International
Electrotechnical
Commission's website](http://www.iec.ch/worldplugs/list_bylocation.htm).

I thought this would be a fun DataSet to use along with Geospatial data so I made sure to incude ISO Country Codes for easy mapping later on. This repository however, will only be dedicated to building a useable list of plug types.

The [worldstandards.eu](http://www.worldstandards.eu/electricity/plugs-and-sockets/) has some good pictures of these plug and socket types.

Here's a link to the CSV World Plugs Data: https://raw.githubusercontent.com/benjiao/world-plugs/master/world-plugs.csv

In [1]:
import requests
response = requests.get("http://www.iec.ch/worldplugs/list_bylocation.htm")

### Parse response

Our HTTP request gives us HTML source code for that page, so we'll need to parse through that to get the data we need. For this purpose, we use BeautifulSoup.

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, 'html.parser')
tbodies = soup.find_all('tbody')

len(tbodies)

1

Great! There is only one tbody on the page. That is probably all we need. Now we check all of the rows from that table:

In [3]:
rows = tbodies[0].find_all('tr')
len(rows)

504

### Inspect data

We can also take a peek at some items, so we have an idea of how each row is probably formatted:

In [4]:
rows[0]

<tr>
<td><div class="flagimg" style="background-image:url(/resources/isoflags/af.png)">
<div class="flagtext">
<p>Afghanistan</p>
</div>
</div></td>
<td><div class="plugimg" style="background-image:url(/worldplugs/img/btn/c-btn-set.png)">
<div class="plugtext">
<p><a href="/worldplugs/typeC.htm" title="Plug Type C: Click to find out more">Type C</a></p>
</div>
</div></td>
<td>220 V</td>
<td>50 Hz</td>
</tr>

In [5]:
rows[1]

<tr>
<td><div class="flagimg" style="background-image:url(/resources/isoflags/af.png)">
<div class="flagtext">
<p>Afghanistan</p>
</div>
</div></td>
<td><div class="plugimg" style="background-image:url(/worldplugs/img/btn/f-btn-set.png)">
<div class="plugtext">
<p><a href="/worldplugs/typeF.htm" title="Plug Type F: Click to find out more">Type F</a></p>
</div>
</div></td>
<td>220 V</td>
<td>50 Hz</td>
</tr>

The data looks great because we already have 1 row per plug-type, per country. One value per column, per row -- we won't have to normalize our data

### Build list of Python dictionaries

In this part, we create a function for building Python dictionary objects out of the HTML formatted file.

In [6]:
def parse_row(row):
    columns = row.find_all("td")
    return {
        'name': columns[0].find('p').string,
        'plug_type': columns[1].find('a').string,
        'voltage': columns[2].string,
        'frequency': columns[3].string
    }

parse_row(rows[0])

{'frequency': '50 Hz',
 'name': 'Afghanistan',
 'plug_type': 'Type C',
 'voltage': '220 V'}

Awesome! Now we can run all of the rows through the function...

In [7]:
rows_dict = [parse_row(row) for row in rows]

### Country Codes

It is also a good idea to map the rows to country codes in case we wan't to use this with other datasets. Here's a function that uses data from [country-codes/master.csv](country-codes/master.csv) and [country-codes/others.csv](country-codes/others.csv).

Take note that I have manually added items missing from [country-codes/master.csv](country-codes/master.csv) into [country-codes/others.csv](country-codes/others.csv).

In [8]:
import csv

def fetch_country_codes():
    with open('country-codes/master.csv', 'r') as f:
        country_codes_master_reader = csv.DictReader(f)
        country_codes = {
            row['Country Name']: row["Country Code"] for row in country_codes_master_reader
        }

    with open('country-codes/others.csv', 'r') as f:
        country_codes_others = csv.DictReader(f)
        for row in country_codes_others:
            country_codes[row['Country Name']] = row["Country Code"]
            
    return country_codes
            
country_codes = fetch_country_codes()

Now we can assign country codes to each row...

In [9]:
for idx, row in enumerate(rows_dict):
    rows_dict[idx]['country_code'] = country_codes[row['name']]

In [10]:
rows_dict[0]

{'country_code': 'AF',
 'frequency': '50 Hz',
 'name': 'Afghanistan',
 'plug_type': 'Type C',
 'voltage': '220 V'}

### Load data into Pandas DataFrame

This will allow us to study our data a bit.

In [11]:
import pandas

country_plugs_df = pandas.DataFrame.from_dict(rows_dict)
country_plugs_df.head()

,country_code,frequency,name,plug_type,voltage
0,AF,50 Hz,Afghanistan,Type C,220 V
1,AF,50 Hz,Afghanistan,Type F,220 V
2,AL,50 Hz,Albania,Type C,230 V
3,AL,50 Hz,Albania,Type F,230 V
4,DZ,50 Hz,Algeria,Type C,230 V


The `count` statistic shown by `describe()` assures us that all rows have no null values. In this case, all columns have counts of 504:

In [12]:
country_plugs_df.describe()

,country_code,frequency,name,plug_type,voltage
count,504,504,504,504,504
unique,218,6,218,14,21
top,MV,50 Hz,Maldives,Type C,230 V
freq,7,238,7,138,191


Just for fun, let's check see socket type is the most widely used:

In [13]:
country_plugs_df.groupby("plug_type")["plug_type"].count().reset_index(name="count").sort_values(['count'], ascending=False)

,plug_type,count
2,Type C,138
5,Type F,74
0,Type A,55
6,Type G,51
1,Type B,47
3,Type D,39
4,Type E,38
8,Type I,20
12,Type M,15
10,Type K,9


I also noticed that Frequency and Voltage sometimes have multiple values per row. However, I have decided to leave it as is, since we are more concerned about the plug types. 

I'll probably build a different dataset for these in the future.

In [14]:
country_plugs_df.groupby("frequency")["frequency"].count().reset_index(name="count").sort_values(['count'], ascending=False)

,frequency,count
3,50 Hz,238
1,50 Hz,164
4,60 Hz,50
5,60 Hz,41
2,"50 Hz, 60 Hz",10
0,60 Hz,1


In [15]:
country_plugs_df.groupby("voltage")["voltage"].count().reset_index(name="count").sort_values(['count'], ascending=False)

,voltage,count
17,230 V,191
13,220 V,153
7,120 V,30
19,240 V,27
18,230 V,22
2,110 V,21
11,"127 V, 220 V",13
10,127 V,7
8,"120 V, 220 V",7
5,115 V,6


### Save to CSV

Let's use Pandas' to_csv function to save the entire DataFrame in CSV format

In [16]:
country_plugs_df.to_csv("world-plugs.csv")